In [1]:
import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time

import pytz

import astroplan
from astroplan import Observer, FixedTarget, ObservingBlock, SequentialScheduler

In [3]:
pal = Observer.at_site('Palomar', timezone=pytz.timezone('US/Pacific'))
pal

<Observer: name='Palomar',
    location (lon, lat, el)=(-116.863 deg, 33.356 deg, 1706.0 m),
    timezone=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>>

In [4]:
targets = [FixedTarget.from_name('Vega'),
           FixedTarget.from_name('Deneb'),
           FixedTarget.from_name('Altair'),
           FixedTarget.from_name('Albireo')]
targets

[<FixedTarget "Vega" at SkyCoord (ICRS): (ra, dec) in deg (279.23473479, 38.78368896)>,
 <FixedTarget "Deneb" at SkyCoord (ICRS): (ra, dec) in deg (310.35797975, 45.28033881)>,
 <FixedTarget "Altair" at SkyCoord (ICRS): (ra, dec) in deg (297.6958273, 8.8683212)>,
 <FixedTarget "Albireo" at SkyCoord (ICRS): (ra, dec) in deg (292.68033548, 27.95968007)>]

In [5]:
etime = 5*u.min
n = 10
rot = 55*u.second

blocks = [ObservingBlock.from_exposures(t, etime, n, rot) for t in targets]

In [6]:
blocks

[<astroplan.scheduling.ObservingBlock object (unscheduled) at 0x10db11610>,
 <astroplan.scheduling.ObservingBlock object (unscheduled) at 0x10c182f10>,
 <astroplan.scheduling.ObservingBlock object (unscheduled) at 0x10c194fd0>,
 <astroplan.scheduling.ObservingBlock object (unscheduled) at 0x10c194c50>]

In [7]:
constraints = [astroplan.constraints.AirmassConstraint(2),
               astroplan.constraints.AtNightConstraint(18*u.deg)
               #,astroplan.constraints.MoonSeparationConstraint(2*u.deg)
               ]

In [8]:
scheduler = SequentialScheduler.from_timespan(pal.midnight(Time.now()), 18*u.hour, 
                                              constraints=constraints, observer=pal)
schedule = scheduler(blocks)
schedule

[<astroplan.scheduling.TransitionBlock object ({u'nothing_observable': <Quantity 30.0 min>}, 2457328.4381 to 2457328.45893) at 0x10c94d410>,
 <astroplan.scheduling.TransitionBlock object ({u'nothing_observable': <Quantity 30.0 min>}, 2457328.45893 to 2457328.47976) at 0x10db08cd0>,
 <astroplan.scheduling.ObservingBlock object (2457328.47976 to 2457328.5006) at 0x10daf3d90>,
 <astroplan.scheduling.ObservingBlock object (2457328.5006 to 2457328.52143) at 0x10c194c90>,
 <astroplan.scheduling.ObservingBlock object (2457328.52143 to 2457328.54226) at 0x10c194bd0>,
 <astroplan.scheduling.ObservingBlock object (2457328.54226 to 2457328.5631) at 0x10c194c10>]

In [9]:
[(pal.astropy_time_to_datetime(b.start_time),pal.astropy_time_to_datetime(b.end_time))  for b in schedule]

[(datetime.datetime(2015, 11, 1, 14, 30, 51, 616859, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>),
  datetime.datetime(2015, 11, 1, 15, 0, 51, 616859, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)),
 (datetime.datetime(2015, 11, 1, 15, 0, 51, 616859, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>),
  datetime.datetime(2015, 11, 1, 15, 30, 51, 616859, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)),
 (datetime.datetime(2015, 11, 1, 15, 30, 51, 616859, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>),
  datetime.datetime(2015, 11, 1, 16, 0, 51, 616859, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)),
 (datetime.datetime(2015, 11, 1, 16, 0, 51, 616859, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>),
  datetime.datetime(2015, 11, 1, 16, 30, 51, 616859, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)),
 (datetime.datetime(2015, 11, 1, 16, 30, 51, 616859, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>),
 

In [10]:
c=  constraints[0]

In [13]:
c(pal, targets, Time.now() + np.linspace(0,3,5)*u.hour).shape

(4, 5)